In [45]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 




In [46]:
import gc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
import lightgbm as lgb
import xgboost as xgb
!pip install catboost
import catboost as cat

In [47]:
LOCAL_QUICK = False
sample_percent = 0.1

MORE_FE = False
# MORE_FE = True
FE_V1 = False if MORE_FE else True

In [48]:
sub_data  = pd.read_csv('data_format1/test_format1.csv')
train_data1 = pd.read_csv('data_format1/train_format1.csv')

user_info = pd.read_csv('data_format1/user_info_format1.csv')
user_log = pd.read_csv('data_format1/user_log_format1.csv')

data_train = pd.read_csv('data_format2/train_format2.csv')

In [49]:
print('All sample train')
data = user_log.copy()
data1 = user_info.copy()
data2 = train_data1.copy()
submission = sub_data.copy() 
del user_log, user_info, train_data1, sub_data
print('---data shape---')     
for df in [data, data1, data2, submission, data_train]:
    print(df.shape)

All sample train
---data shape---
(331619, 7)
(424170, 3)
(260864, 3)
(261477, 3)
(7030723, 6)


In [50]:
data2.sample()

,user_id,merchant_id,label
79400,378474,361,0


In [51]:
submission.sample()

,user_id,merchant_id,prob
190860,332975,1923,NaN


In [52]:
data1.sample()

,user_id,age_range,gender
175662,186457,3.0,1.0


In [53]:
data.sample()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
76644,392237,138700,756,2619,8022.0,1110.0,0.0


In [54]:
data2['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([data2, submission], ignore_index=True, sort=False)
matrix.drop(['prob'], axis=1, inplace=True)

In [55]:
matrix.shape

(522341, 4)

In [56]:
matrix.sample()

,user_id,merchant_id,label,origin
293635,310752,227,NaN,test


In [57]:
matrix.label.value_counts()

0.0    244912
1.0     15952
Name: label, dtype: int64

In [58]:
matrix.label.isna().sum()

261477

In [59]:
matrix = matrix.merge(data1, on='user_id', how='left')
# 使用merchant_id（原列名seller_id）
data.rename(columns={'seller_id':'merchant_id'}, inplace=True)

In [60]:
data['user_id'] = data['user_id'].astype('int32')
data['merchant_id'] = data['merchant_id'].astype('int32')
data['item_id'] = data['item_id'].astype('int32')
data['cat_id'] = data['cat_id'].astype('int32')
data['brand_id'].fillna(0, inplace=True)
data['brand_id'] = data['brand_id'].astype('int32')

In [61]:
data.sample()

,user_id,item_id,cat_id,merchant_id,brand_id,time_stamp,action_type
98542,118993,9442,602,8,3731,1018.0,0.0


In [62]:
data['time_stamp'] = pd.to_datetime(data['time_stamp'], format='%H%M')

In [63]:
# 缺失值填充
matrix['age_range'].fillna(0, inplace=True)
matrix['gender'].fillna(2, inplace=True)

matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')

del data1, data2
gc.collect()

189

In [64]:
##### 特征处理
##### User特征处理
groups = data.groupby(['user_id'])

In [65]:
temp = groups.size().reset_index().rename(columns={0:'u1'})
temp.sample()

,user_id,u1
58,10450,115


In [66]:
matrix = matrix.merge(temp, on='user_id', how='left')

In [67]:
# 其实就是和下面的作用一样
groups['item_id'].agg([('u2', 'nunique')]).reset_index()

,user_id,u2
0,16,36
1,281,43
2,330,86
3,342,9
4,445,181
...,...,...
2245,423060,44
2246,423575,56
2247,423731,602
2248,423921,91


In [68]:
# 该过程写法更容易理解 
data.groupby(['user_id']).agg({'item_id':'nunique'}).reset_index()

,user_id,item_id
0,16,36
1,281,43
2,330,86
3,342,9
4,445,181
...,...,...
2245,423060,44
2246,423575,56
2247,423731,602
2248,423921,91


In [69]:
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
# 时间间隔特征 u6 按照小时
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')

In [70]:
# 统计操作类型为0，1，2，3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(  
    columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')

del temp
gc.collect()

15

In [71]:
##### 商家特征处理
groups = data.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={
    'user_id':'m2',
    'item_id':'m3', 
    'cat_id':'m4', 
    'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(  
    columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

del temp
gc.collect()

0

In [72]:
# 按照merchant_id 统计随机负采样的个数
temp = data_train[data_train['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

In [73]:
matrix.head(5)

,user_id,merchant_id,label,origin,age_range,gender,u1_x,u1_y,u3,u4,...,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10
0,34176,3906,0.0,train,6,0,NaN,NaN,NaN,NaN,...,217.0,35.0,74.0,15.0,2.0,186.0,1.0,10.0,20.0,2861
1,34176,121,0.0,train,6,0,NaN,NaN,NaN,NaN,...,841.0,95.0,369.0,17.0,1.0,785.0,NaN,36.0,20.0,4530
2,34176,4356,1.0,train,6,0,NaN,NaN,NaN,NaN,...,31.0,13.0,5.0,4.0,1.0,26.0,NaN,3.0,2.0,1088
3,34176,2217,0.0,train,6,0,NaN,NaN,NaN,NaN,...,197.0,76.0,74.0,4.0,1.0,174.0,NaN,8.0,15.0,7268
4,230784,4818,0.0,train,0,0,NaN,NaN,NaN,NaN,...,219.0,31.0,75.0,16.0,2.0,197.0,2.0,12.0,8.0,3102


In [74]:
##### 用户+商户特征

groups = data.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={
    'item_id':'um2',
    'cat_id':'um3',
    'brand_id':'um4'
})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={
    0:'um5',
    1:'um6',
    2:'um7',
    3:'um8'
})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('frist', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['frist']).dt.seconds/3600
temp.drop(['frist', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

del temp
gc.collect()

15

In [75]:
matrix['r1'] = matrix['u9']/matrix['u7'] # 用户购买点击比
matrix['r2'] = matrix['m8']/matrix['m6'] # 商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5'] #不同用户不同商家购买点击比

In [76]:
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

del temp
gc.collect()

30

In [77]:
# train、test-setdata
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)

if not LOCAL_QUICK:
    if FE_V1:
        train_data.to_csv('train_data.csv')
        test_data.to_csv('test_data.csv')
    if MORE_FE:
        train_data.to_csv('train_data_moreFE.csv')
        test_data.to_csv('test_data_moreFE.csv')

del matrix
gc.collect()

0

## 导入数据

In [78]:
# get data
if not LOCAL_QUICK:
    if FE_V1:
        train_data = pd.read_csv('train_data.csv')
        test_data = pd.read_csv('test_data.csv')
    if MORE_FE:
        train_data = pd.read_csv('train_data_moreFE.csv')
        test_data = pd.read_csv('test_data_moreFE.csv')  

# FeatureSelect_QUICK = True # Feature Select
FeatureSelect_QUICK = False 
if FeatureSelect_QUICK: # 使用部分样本进行快速特征选择
    train_data = train_data.sample(int(len(train_data) * sample_percent))

# train_data = train_data[train_col]
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

del train_data

X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2, random_state=42) # test_size=.3

### xgboost

In [80]:
def xgb_train(X_train, y_train, X_valid, y_valid, verbose=True):
    model_xgb = xgb.XGBClassifier(
        max_depth=10, # raw8
        n_estimators=1000,
        min_child_weight=300, 
        colsample_bytree=0.8, 
        subsample=0.8, 
        eta=0.3,    
        seed=42        
    )

    model_xgb.fit(
        X_train, 
        y_train,
        eval_metric='auc',
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        verbose=verbose,
        early_stopping_rounds=10 # 早停法，如果auc在10epoch没有进步就stop
    )
    print(model_xgb.best_score)
    return model_xgb

In [81]:
model_xgb = xgb_train(X_train, y_train, X_valid, y_valid, verbose=False)

0.640567


In [82]:

%%time
prob = model_xgb.predict_proba(test_data)

submission['prob'] = pd.Series(prob[:,1])
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('submission_xgb.csv', index=False)

CPU times: user 2.99 s, sys: 91 ms, total: 3.09 s
Wall time: 571 ms


### LGB 模型

In [83]:
def lgb_train(X_train, y_train, X_valid, y_valid, verbose=True):
    model_lgb = lgb.LGBMClassifier(
        max_depth=10, # 8
        n_estimators=1000,
        min_child_weight=200, 
        colsample_bytree=0.8, 
        subsample=0.8, 
        eta=0.3,    
        seed=42        
    )

    model_lgb.fit(
        X_train, 
        y_train,
        eval_metric='auc',
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        verbose=verbose,
        early_stopping_rounds=10
    )

    print(model_lgb.best_score_['valid_1']['auc'])
    return model_lgb

In [84]:
model_lgb = lgb_train(X_train, y_train, X_valid, y_valid, verbose=False)

[LightGBM] [Warning] learning_rate is set=0.1, eta=0.3 will be ignored. Current value: learning_rate=0.1


LightGBMError: Do not support special JSON characters in feature name.

In [ ]:
submission['prob'] = pd.Series(prob[:,1])
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('submission_lgb.csv', index=False)

### Catboost

In [85]:
def cat_train(X_train, y_train, X_valid, y_valid, verbose=True):
    model_cat = cat.CatBoostClassifier(learning_rate=0.02, iterations=5000, eval_metric='AUC', od_wait=50,
                                od_type='Iter', random_state=10, thread_count=8, l2_leaf_reg=1, verbose=verbose)
    model_cat.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=50,
            use_best_model=True)

    print(model_cat.best_score_['validation']['AUC'])
    return model_cat

In [86]:
model_cat = cat_train(X_train, y_train, X_valid, y_valid, verbose=False)

0.6456994227209207


In [87]:

%%time
prob = model_cat.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('submission_cat.csv', index=False)

CPU times: user 763 ms, sys: 56.7 ms, total: 820 ms
Wall time: 476 ms
